In [121]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
import time

In [122]:
def click_mas():
    # Pinchar en el botón del menu "Más"
    masMenu = driver.find_element(By.XPATH, '//*[@id="content"]/header/div[2]/ul/li[5]/a')

    try:
        masMenu.click()
    except (ElementNotInteractableException, NoSuchElementException):
        # Maneja la excepción y espera antes de intentar nuevamente
        print("Anuncio detectado, reiniciando driver...")
        driver.refresh()
        time.sleep(3) 
        masMenu.click()

In [123]:
# Función para obtener el valor basado en la etiqueta
def obtener_valor_por_etiqueta(label_deseado):
    elemento = driver.find_element(By.XPATH, f"//div[@class='item']//div[@class='label' and text()='{label_deseado}']/following-sibling::div[@class='value']")
    valor = elemento.text
    return valor

In [124]:
teams_data = {
    "Real Madrid": "https://cdn.gomister.com/file/cdn-common/teams/15.png?version=20231117",
    "Real Sociedad": "https://cdn.gomister.com/file/cdn-common/teams/16.png?version=20231117",
    "Atlético de Madrid": "https://cdn.gomister.com/file/cdn-common/teams/2.png?version=20231117",
    "Girona": "https://cdn.gomister.com/file/cdn-common/teams/222.png?version=20231117",
    "Osasuna": "https://cdn.gomister.com/file/cdn-common/teams/50.png?version=20231117",
    "Athletic Club": "https://cdn.gomister.com/file/cdn-common/teams/1.png?version=20231117",
    "Valencia": "https://cdn.gomister.com/file/cdn-common/teams/19.png?version=20231117",
    "Granada": "https://cdn.gomister.com/file/cdn-common/teams/10.png?version=20231117",
    "Getafe": "https://cdn.gomister.com/file/cdn-common/teams/9.png?version=20231117",
    "Villarreal": "https://cdn.gomister.com/file/cdn-common/teams/20.png?version=20231117",
    "Las Palmas": "https://cdn.gomister.com/file/cdn-common/teams/11.png?version=20231117",
    "Mallorca": "https://cdn.gomister.com/file/cdn-common/teams/408.png?version=20231117",
    "Rayo Vallecano": "https://cdn.gomister.com/file/cdn-common/teams/14.png?version=20231117",
    "Barcelona": "https://cdn.gomister.com/file/cdn-common/teams/3.png?version=20231117",
    "Celta de Vigo": "https://cdn.gomister.com/file/cdn-common/teams/5.png?version=20231117",
    "Cádiz": "https://cdn.gomister.com/file/cdn-common/teams/499.png?version=20231117",
    "Alavés": "https://cdn.gomister.com/file/cdn-common/teams/48.png?version=20231117",
    "Almería": "https://cdn.gomister.com/file/cdn-common/teams/21.png?version=20231117",
    "Sevilla": "https://cdn.gomister.com/file/cdn-common/teams/17.png?version=20231117",
    "Betis": "https://cdn.gomister.com/file/cdn-common/teams/4.png?version=20231117",
}

In [125]:
def actualizar_version(teams_data, version):
      for equipo, url in teams_data.items():
        # Dividir la URL en base al signo de interrogación
        partes = url.split('?')
        
        # Verificar si hay una parte después del signo de interrogación y actualizar la versión
        if len(partes) > 1:
            partes[1] = f"version={version}"
            
            # Volver a unir las partes para formar la URL actualizada
            nueva_url = '?'.join(partes)
            
            # Actualizar la URL en el diccionario
            teams_data[equipo] = nueva_url

        #print(version)
        #print("nuevaaaa url-->  ",nueva_url)

In [208]:
def extraer_info_jugador():
    
    nombre = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[2]")
    apellido = driver.find_element(By.XPATH, " /html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[3]")
    valor=driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div[2]/div/div/div[1]/div[2]')
        
    media_puntos_local = obtener_valor_por_etiqueta("Media en casa")
    media_puntos_visitante = obtener_valor_por_etiqueta("Media fuera")
    try:
        edad = obtener_valor_por_etiqueta("Edad")
        altura = obtener_valor_por_etiqueta("Altura")
        peso = obtener_valor_por_etiqueta("Peso")
    except:
        edad = None
        altura = None
        peso = None
            
    if peso == "kg":
        peso = None
        
        
    #OBTENER EQUIPO JUGADOR
    
    # Obtener src del equipo
    team_logo_element = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[1]/a/img")
    image_url = team_logo_element.get_attribute("src")


    # Comparar la URL de la imagen con las URLs en teams_data
    equipo = None
    proximo_rival=None
    local= False
    for equipo_nombre, equipo_url in teams_data.items():
        if image_url == equipo_url:
            equipo = equipo_nombre
            
            #OBTENER POSICIÓN DEL JUGADOR 
            try:
                divpartido = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div[1]/div[3]/div")
            except:
                divpartido = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div/div[2]/div")
            
            # Encuentra el div del partido
            item_elements = divpartido.find_elements(By.CLASS_NAME, 'item')
           
            # Encuentra las imágenes dentro del div partido
            img_elements = item_elements[0].find_elements(By.CLASS_NAME, 'team-logo')

            # Guarda las src de las imágenes en variables
            if len(img_elements) >= 2:
                src_img1 = img_elements[0].get_attribute('src')
                src_img2 = img_elements[1].get_attribute('src')
                if src_img1 == image_url:
                    local = True
                    for equipo_nombre, equipo_url in teams_data.items():
                        if src_img2 == equipo_url:
                            proximo_rival=equipo_nombre
                else:
                    local=False
                    for equipo_nombre, equipo_url in teams_data.items():
                        if src_img1 == equipo_url:
                            proximo_rival=equipo_nombre
            else:
                print("No se encontro el próximo partido")
            

            
     #OBTENER POSICIÓN DEL JUGADOR

    elemento = driver.find_element(By.XPATH, '//i[contains(@class, "pos-")]')
    # Obtener el valor del atributo class
    clases = elemento.get_attribute("class").split()

    # Determinar la posición
    posicion = None
    for clase in clases:
        if clase.startswith("pos-") and "pos-big" in clases:
            if clase == "pos-1":
                posicion = "PT"
            elif clase == "pos-2":
                posicion = "DF"
            elif clase == "pos-3":
                posicion = "MC"
            elif clase == "pos-4":
                posicion = "DL"
            break
            
            
    #OBTENER PUNTOS DEL JUGADOR
    
    # Encontrar jornada
    elemento_principal = driver.find_element(By.CLASS_NAME, 'btn-player-gw')
    subelemento_gw = elemento_principal.find_element(By.CLASS_NAME, 'gw')

    #print(subelemento_gw.text)

    # Encontrar jornada en la web con otro elemennto como referencia
    localizador = driver.find_element(By.XPATH, "//h4[text()='Valor']")
    driver.execute_script("arguments[0].scrollIntoView(true);", localizador)   
  
    time.sleep(1)
    try:
        subelemento_gw.click()
    except:
        elemento_principal.click()
    
    time.sleep(2)
    try:
        # PUNTOS MISTER FANTASY
        main_provider = driver.find_element(By.CLASS_NAME, 'main-provider')
        points_element = main_provider.find_element(By.CLASS_NAME, 'points')
        final_points = points_element.get_attribute('data-points')
        
        # PUNTOS AS, MARCA Y MUNDO DEPORTIVO 
        providers_div = driver.find_element(By.CLASS_NAME, "providers")
        li_elements = providers_div.find_elements(By.TAG_NAME, "li")

        points_array = []

        for li_element in li_elements:
           
            points_div = li_element.find_element(By.CLASS_NAME, "points")
            points_value = points_div.text
            points_array.append(points_value)
    
        as_points=points_array[0]
        marca_points=points_array[1]
        mundo_deportivo_points=points_array[2]

        driver.back()
    except:
        final_points=None
        as_points=None
        marca_points=None
        mundo_deportivo_points=None
    
    
    
    #IMPRIMIR TODOS LSO DATOS
    print("____________________________________")
    print(index," -",nombre.text,apellido.text)
    print("Valor: ",valor.text)
    print("Posición: ",posicion)
    print("Equipo: ",equipo)
    
    print("- - - - - - - - - - - - - - - - - -")
   
    print("Puntuación Fantasy:", final_points)
    print("Puntuación Fantasy:", as_points)
    print("Puntuación Marca:", marca_points)
    print("Puntuación Mundo Deportivo:", mundo_deportivo_points)
 
    print("- - - - - - - - - - - - - - - - - -")
    
    print("Próximo rival: ",proximo_rival)
    print("Próximo partido es local: ",local)
    print(f"Media en casa: {media_puntos_local}")
    print(f"Media fuera: {media_puntos_visitante}")
    print(f"Edad: {edad}")
    print(f"Altura: {altura}")
    print(f"Peso: {peso}")

In [209]:
# Crea una instancia del controlador del navegador
driver = webdriver.Chrome()

# Navega a la página web que deseas hacer scraping
driver.get("https://mister.mundodeportivo.com/new-onboarding/#market")

# Espera a que se cargue la página
driver.implicitly_wait(15)

# Encuentra el botón de "Consentir" 
button = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')
# Haz clic en el botón de "Consentir" 
button.click()

# Encuentra el botón de "Siguinete" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[2]/button')
# Haz clic en el botón de "Siguiente" 
button.click()
button.click()
button.click()
button.click()

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/button[3]')
button.click()

# Localiza el elemento del input gmail
inputgmail = driver.find_element(By.XPATH, '//*[@id="email"]')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputgmail.clear()

# Ingresa texto en la caja de texto
inputgmail.send_keys("m31_grupo6@outlook.com")

# Localiza el elemento del input gmail
inputpsw = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[2]/input')

# Borra cualquier contenido existente en la caja de texto (opcional)
inputpsw.clear()

# Ingresa texto en la caja de texto
inputpsw.send_keys("Chocoflakes2")

# Encuentra el botón de "sing con gmail" 
button = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div/form/div[3]/button')
button.click()

# Espera a que se cargue la página
driver.implicitly_wait(10)

click_mas()

# Pinchar en el botón "Jugaodres" para acceder al listado de jugadores 
jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')

try:
    jugadoresbtn.click()
except (ElementNotInteractableException, NoSuchElementException):
    # Maneja la excepción y espera antes de intentar nuevamente
    print("Anuncio detectado, reiniciando driver...")
    driver.refresh()
    time.sleep(3)
    click_mas()
    time.sleep(3)
    try:
        jugadoresbtn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/button[2]')
        jugadoresbtn.click()
    except: 
        print("Reicicia el script :(")
        sys.exit()
             
pag=2
index=0
while True:

    # Encontrar todos los elementos li
    elementos_lis = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")

    # Longitud de la lista de elementos encontrados
    length=len(elementos_lis)

    while index < length:
        
        # Encontrar todos los elementos li
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        elementos_li[index].click()

        time.sleep(1)
        
        try:
            team_logo_element = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[2]/div[1]/div/div[1]/div[1]/a/img")
        except:
            try:
                team_logo_element = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div/div[3]/div/div[1]/div[2]/img[1]")
            except:
                team_logo_element = driver.find_element(By.XPATH, "/html/body/div[6]/div[3]/div[3]/div/div[3]/div/div[1]/div[2]/img[2]")
        
        image_url = team_logo_element.get_attribute("src")
        # Dividir la URL utilizando el signo de igual como delimitador
        parts = image_url.split('=')
        # El valor de version está en la segunda parte después del =
        version = parts[1]
        
        actualizar_version(teams_data, version)
        extraer_info_jugador()
        
        #Retroceder página
        driver.back()
        time.sleep(1)
        elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
        if index == 0:
            driver.execute_script("arguments[0].scrollIntoView(true);", elementos_li[index])
        else:
            driver.execute_script("arguments[0].scrollIntoView(true);", elementos_li[index-1])
        time.sleep(1)
        index += 1

    #Pulsar Ver más
    try:
        ver_mas = driver.find_element(By.XPATH, '/html/body/div[6]/div[3]/div[3]/div[1]/button')
        ver_mas.click()
        time.sleep(4)
    except:
        break

    #Jugador cambio de pagina
    elementos_li = driver.find_elements(By.XPATH, "/html/body/div[6]/div[3]/div[3]/ul/li")
    elementos_li[index].click()
    time.sleep(2)
    extraer_info_jugador()
    driver.back()
    
    print("____________________________________")
    print("------------------------------------")
    print("Sigueinte página...  (",pag,")")
    print("------------------------------------")
    
    
    index=1
    pag+=1

driver.quit()    
print("Todos los jugadores scapeados")

Anuncio detectado, reiniciando driver...
____________________________________
0  - Jude Bellingham
Valor:  22.016.000
Posición:  MC
Equipo:  Real Madrid
- - - - - - - - - - - - - - - - - -
Puntuación Fantasy: None
Puntuación Fantasy: None
Puntuación Marca: None
Puntuación Mundo Deportivo: None
- - - - - - - - - - - - - - - - - -
Próximo rival:  Cádiz
Próximo partido es local:  False
Media en casa: 10,3
Media fuera: 12,3
Edad: 20
Altura: 1,88m
Peso: None
____________________________________
1  - Takefusa Kubo
Valor:  20.039.000
Posición:  MC
Equipo:  Real Sociedad
- - - - - - - - - - - - - - - - - -
Puntuación Fantasy: 3
Puntuación Fantasy: 2
Puntuación Marca: 2
Puntuación Mundo Deportivo: 2
- - - - - - - - - - - - - - - - - -
Próximo rival:  Sevilla
Próximo partido es local:  True
Media en casa: 10,6
Media fuera: 5,2
Edad: 22
Altura: 1,74m
Peso: 67kg
____________________________________
2  - Antoine Griezmann
Valor:  22.429.000
Posición:  DL
Equipo:  Atlético de Madrid
- - - - - - - - 